# Réseaux de Neurones Récurrents (RNN & LSTM)

Les réseaux de neurones récurrents sont conçus pour traiter des **données séquentielles** comme le texte, la parole ou les séries temporelles. 
 
Ils exploitent l’ordre des éléments dans une séquence.

## 1. Réseaux de Neurones Récurrents (RNN)

Un **RNN** traite les données mot par mot (ou token par token) en conservant une **mémoire** de ce qui a été vu précédemment.

- La sortie dépend de l’entrée courante et de l’état précédent

- Adapté aux séquences de longueur variable

**Limites**

- Difficulté à apprendre des dépendances longues

- Problème de gradient qui disparaît (vanishing gradient)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=10000)
print(f"Train samples: {len(input_train)}")
print(f"Exemple d'avis (sous forme d'index): {input_train[0][:10]}...")

# mettre les données à la même echelle
maxlen = 200
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print(f"Shape des données d'entrée : {input_train.shape}")

In [ ]:
class IMDbDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = torch.LongTensor(sequences)
        self.labels = torch.FloatTensor(labels)
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

train_dataset = IMDbDataset(input_train, y_train)
test_dataset = IMDbDataset(input_test, y_test)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    train_dataset, [train_size, val_size]
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
class SentimentRNN(nn.Module):
    def __init__(self, vocab_size=10000, embedding_dim=32, hidden_dim=32):
        super(SentimentRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        embedded = self.embedding(x)
        _, h_n = self.rnn(embedded)
        h_n = h_n.squeeze(0) 
        out = self.fc(h_n)
        out = self.sigmoid(out)
        return out.squeeze(1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nUtilisation de : {device}")

model = SentimentRNN().to(device)

print("\nArchitecture du modèle :")
print(model)
print(f"\nNombre de paramètres : {sum(p.numel() for p in model.parameters()):,}")

criterion = nn.BCELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for sequences, labels in loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    return total_loss / len(loader), correct / total


def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for sequences, labels in loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    return total_loss / len(loader), correct / total

In [ ]:
epochs = 10

for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f} - Train Acc: {train_acc:.2%}")
    print(f"  Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.2%}")

test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"\nAccuracy sur le Test Set : {test_acc:.2%}")